In [1]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.spatial import distance_matrix
import sys
import numpy as np

from train import Training

In [2]:
params_list = {"model_type": "GCN",  
               "n_graph_subsampling": 0, # the number of running graph subsampling each train graph data run subsampling 5 times: increasing graph data 5 times
               "graph_node_subsampling": True, # TRUE: removing node randomly to subsampling and augmentation of graph dataset \n'+
                # FALSE: removing edge randomly to subsampling and augmentation of graph dataset
               "graph_subsampling_rate": 0.2, # graph subsampling rate
               "dataset": "IMDB", 
               "pooling_type": "mean", 
               "seed": 42,
               "n_folds": 3, 
               "cuda": True, 
               "lr": 0.001, 
               "epochs": 20, 
               "weight_decay":5e-4,
               "batch_size": 32, 
               "dropout": 0, # dropout rate of layer
               "num_lay": 5, 
               "num_agg_layer": 2, # the number of graph aggregation layers
               "hidden_agg_lay_size": 64, # size of hidden graph aggregation layer
               "fc_hidden_size": 128, # size of fully-connected layer after readout
               "threads":10, # how many subprocesses to use for data loading
               "random_walk":True,
               "walk_length": 20, # walk length of random walk, 
               "num_walk": 10, # num of random walk
               "p": 0.65, # Possibility to return to the previous vertex, how well you navigate around
               "q": 0.35, # Possibility of moving away from the previous vertex, how well you are exploring new places
               "print_logger": 10  # printing rate
               }

In [3]:
trainer = Training(params_list)

data path: data/IMDB-BINARY/raw/IMDB-BINARY
complete to build adjacency matrix list
complete to build node count list
complete to build edge matrix list
complete to build edge matrix count list
complete to build max neighbor list
node label: degree of nodes
complete to build node features list
complete to build targets list
--------------------------------------------------
The number of graphs: 1000
N nodes avg/std/min/max: 	19.77/10.06/12/136
N edges avg/std/min/max: 	96.53/105.60/26/1249
Node degree avg/std/min/max: 	9.76/7.43/1/135
Node features dim: 		135
N classes: 			2
Classes: 			[0 1]
Class 0: 			500 samples
Class 1: 			500 samples
feature 1, count 1/19773
feature 2, count 30/19773
feature 3, count 790/19773
feature 4, count 2144/19773
feature 5, count 3275/19773
feature 6, count 2730/19773
feature 7, count 2444/19773
feature 8, count 1118/19773
feature 9, count 861/19773
feature 10, count 554/19773
feature 11, count 1140/19773
feature 12, count 427/19773
feature 13, count 258

In [5]:
x = trainer.datareader.data

In [14]:
trainer.fit()

AttributeError: 'Training' object has no attribute 'fit'